In [1]:
import xarray
import gsw
import numpy as np
import matplotlib.pyplot as plt
import math

<h3>Lattitude from -89.5 to 89.5 with 0.5 difference </h3>
<h3>Longitude from -179.5 to 179.5 with 0.5 difference</h3>

In [2]:
lat_array = np.linspace(-89.75, 89.75, 360)     
lon_array = np.linspace(-179.75, 179.75, 720)

<h3> Parameters of Fourier series model (FSM) SLA </h3>

In [3]:
omega = 7.2921*(10**-5)     #Angular speed of Earth

f=2*omega*np.sin(np.deg2rad(lat_array))    #Corolis Parameter
g = 9.8             #Acceleration due to gravity
radius = 6370e3     #Radius of Earth


<h3> dy and dx of Fourier series model </h3>

In [4]:
dlat = 2*np.pi*(0.5/360)*radius       
dlon = 2*np.pi*radius*(0.5/360)*np.cos(np.deg2rad(lat_array)).reshape(360,1)* np.ones((360,720))  


<h3>We are taking 5 years data of every months and storing in a variable  </h3>

In [7]:
data_array = []
steps = (slice(None, None, 15), slice(None, None, 15))     #spacing between Arrows

#Combining 5 year data in one list

for year in range(2010,2015):
    for month in range(1,13):
        if month<10:
            mon = '0'+str(month)
        else:
            mon = str(month)
        data = xarray.open_dataset('SEA_SURFACE_HEIGHT_mon_mean_{y}-{m}_ECCO_V4r4_latlon_0p50deg.nc'.format(y=year,m=mon))
        new_data = data.loc[dict(longitude=slice(-180,180), latitude=slice(-90,90))].SSH
        data_array.append(new_data[0])
data_arr=np.array(data_array)

<h3> getting average of each months of 5 years </h3>


In [8]:
for i in range(1,5):
    if i==1:
        yearly_sum = data_arr[12*(i-1):12*(i),:,:]
    else:
        yearly_sum = yearly_sum + data_arr[12*(i-1):12*(i),:,:]
        
yearly_avg = yearly_sum/5

<h3>  Lattitude and Longitude for Graph Axis </h3>

In [9]:
lat_grid = np.linspace(-89.75, 89.75, 360).reshape(360,1)* np.ones((360,720))
lon_grid = np.linspace(-179.75, 179.75, 720).reshape(1,720)* np.ones((360,720))


<h3> differnce in the sea surface height along lattitude and longitude (∂ζ)  </h3>

In [10]:
for i in range (2):
 yav_data=yearly_avg[i]

 reshaped_arr = yav_data[:,0].reshape(len(yav_data[:,0]),1)
 lon_data_proc = np.concatenate((yav_data, reshaped_arr), axis=1)
 lon_diff_data = np.diff(lon_data_proc, axis=1)
 print(lon_data_proc.shape)


 lat_data_proc = np.concatenate((yav_data, yav_data[0,:].reshape(1,len(yav_data[0,:]))), axis=0)
 lat_diff_data = np.diff(lat_data_proc, axis=0)


(360, 721)
(360, 721)


<h3> surface geostrophic current
  </h3>

In [11]:
 ug = (-g/f.reshape(len(f),1))*(lat_diff_data/dlat)   # ug = (−g/f)(∂ζ/∂y)
 vg = (g/f.reshape(len(f),1))*(lon_diff_data/dlon)    # vg = (g/f)(∂ζ/∂x)


<h3>Magnitude of ug and vg </h3>

<h3>Normalization of ug and vg </h3>

In [12]:
 magnitude_arr = np.sqrt(ug**2 + vg**2)
 magnitude_arr[magnitude_arr == 0] = 'nan'
 
 ug_unit = ug/magnitude_arr
 vg_unit = vg/magnitude_arr

<h3>Ploting the countor plot  </h3>


In [14]:
plt.figure(figsize=(14,8))
level=np.linspace(-1.5,1,20)



plt.contourf(lon_grid, lat_grid, yav_data,levels=level,,cmap=plt.get_cmap('hsv')) 
plt.colorbar()



plt.quiver(lon_grid[steps], lat_grid[steps],
                 vg_unit[steps], ug_unit[steps],
                 color='black')
for i in range(360):
    for j in range (720):
        if(i==359 or j==719):
            ug[i,j]=math.nan
            vg[i,j]=math.nan

plt.show()



SyntaxError: invalid syntax (3448060470.py, line 6)